# 트리의 앙상블

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 랜덤포레스트

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate # 교차 검증
from sklearn.ensemble import RandomForestClassifier # 랜덤포레스트 클래스

rf = RandomForestClassifier(n_jobs=-1, random_state=42) # 객체 생성(모든cpu 코어 사용)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1) # 모델 객체 교차 검증, 검증세트 뿐만 아니라 훈련세트 점수 반환

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 훈련세트 과대적합 

0.9973541965122431 0.8905151032797809


In [ ]:
rf.fit(train_input, train_target) 
print(rf.feature_importances_) # 특성 중요도가 결정트리때 보다 표준편차가 완화된 것으로 보임>> 과대적합을 줄이고 성능을 높임

[0.23167441 0.50039841 0.26792718]


In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_) # 중복과 랜덤으로 부트스트랩 샘플에 포함되지 않은 OOB샘플의 평균점수 >> 교차 검증에서 얻은 점수와 비슷한 결과

0.8934000384837406


## 엑스트라트리

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier # 엑스트라트리는 부스트샘플을 사용하지 않음, 대신 노드를 분할 할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 랜덤 포레스트와 비슷한 결과

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_) # 특성 중요도 확인(도수,농도,ph)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier # 깊이가 얕은 결정 트리를 사용>깊이가 3인 결정트리 100개 사용 

gb = GradientBoostingClassifier(random_state=42) 
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)  # 결정트리개수를 500개로 늘리고, 학습률을 디폴트0.1>>0.2로 변경 
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1) # 모델 교차검증

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 이전보다 과대적합을 잘 억제함

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_) # 특성 중요도에 당도 비율이 높아짐

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 부스팅

In [ ]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier # 히스토그램 부스팅

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 과대적합을 잘 억제하고 성능이 조금 ㄷ 좋아짐

0.9321723946453317 0.8801241948619236


In [ ]:
from sklearn.inspection import permutation_importance # 특성 중요도를 계산하기 위한 클래스 함수

hgb.fit(train_input, train_target) 
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, # 훈련세트 중요도 확인 # n_repeats매개변수는 랜덤하게 섞을 횟수 지정
                                random_state=42, n_jobs=-1)
print(result.importances_mean) # 랜덤 포레스트와 비슷한 비율

[0.08876275 0.23438522 0.08027708]


In [ ]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,# 테스트세트 중요도 확인
                                random_state=42, n_jobs=-1)
print(result.importances_mean) # 그레이디언트 부스팅과 비슷한 비율(랜덤보다 높은 당도 비율)

[0.05969231 0.20238462 0.049     ]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

#### XGBoost

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


#### LightGBM

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


* 양상블 학습: 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 머신러닝 알고리즘

* 랜덤 포레스트: 대표적인 결정 트리 기반의 양상블 학습 방법, 부트스트랩 샘플을 사용하고 랜덤하게 일부 특성을 선택하여 트리를 만듬

* 엑스트라 트리: 결정 트리를 사용하여 양상블 모델을 만들지만 부트스트랩 사용을 하지 않고, 랜덤하게 노드를 분할해 과대적합을 감소시킴

* 그레이디언트 부스팅: 결정 트리를 연속적으로 추가하여 손실 함수를 최소화하는 양상블 방법, 속도는 느리지만 좋은 성능을 기대할 수 있고 속도를 개선한 것이 히스토그램 기반 그레이디언트 부스팅임

* scikit-learn
  - RandomForestClassifier 랜덤 포레스트 분류 클래스
    * n_estimators 매개변수는 트리의 개수를 지정
    * criterion 매개변수는 불순도를 지정
    * max_depth는 트리가 성장할 최대 깊이를 지정
    * min_samples_sploit은 노드를 나누기 위한 최소 샘플 개수, 기본값=2
    * max_features 매개변수는 최적의 분할을 위해 탐색할 특성의 개수를 지정
    * bootstrap 매개변수는 부트스트랩 샘플을 사용할지 지정
    * oob_score는 OOB 샘플을 사용하여 훈련한 모델을 평가할지 지정
    * n_job 매개변수는 벙렬 실행에 사용할 cpu 코어 수를 지정

  - ExtraTreesClassifier 엑스트라 트리 분류 클래스
    * bootstrap 매개변수 기본값 = False
    * oob_scoreㄴ 매개변수 기본값 = False
    * n_jobs 매개변수는 cpu 코어수 지정 기본값=1, 모든코어는 -1

  - GradientBoostingClassifier 그레이디언트 부스팅 분류 클래스
    * loss 매개변수는 손실 함수 지정
    * learning_rate 매개변수는 트리가 양상블에 기여하는 정도
    * n_estimators 매개변수는 부스팅 단계를 수행하는 트리의 개수
    * subsample 매개변수는 사용할 훈련 세트의 샘플 비율을 지정
    * max_depth 매개변수는 트리의 최대 깊이

  - HistGradientBoostringClassifier 히스토그램 기반 그레이디언트 부스팅 분류 클래스
    * learning_rate 매개변수는 학습률이라고 함 기본값=0.1
    * max_iter 부스팅 단계를 수행하는 트리의개수
    * max_bins 입력데이터를 나눌 구간의 개수
